# Dendritic Spine Clusterization

In [ ]:
from spine_analysis.spine.grouping import SpineGrouping
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import SpineMeshDataset, intersection_ratios_mean_distance
from spine_segmentation import apply_scale


dataset_path = "0.025 0.025 0.1 dataset"
scale = (1, 1, 1)

# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_path)
spine_dataset.apply_scale(scale)

# load merged and reduced manual classification
manual_classification = SpineGrouping().load(f"{dataset_path}/manual_classification/manual_classification_merged_reduced.json")

# load metrics
spine_metrics = SpineMetricDataset().load(f"{dataset_path}/metrics.csv")
spine_metrics = spine_metrics.get_spines_subset(manual_classification.samples)

# extract metric subsets
classic = spine_metrics.get_metrics_subset(['OpenAngle', 'CVD', "JunctionArea", 'AverageDistance', 'Length', 'Area', 'Volume', 'ConvexHullVolume', 'ConvexHullRatio', "LengthVolumeRatio", "LengthAreaRatio"])
classic_prefix = f"{dataset_path}_classic"
chord = spine_metrics.get_metrics_subset(['OldChordDistribution'])
chord_prefix = f"{dataset_path}_chord"

# set score function to mean distance between class over cluster distributions
score_func = lambda clusterizer: intersection_ratios_mean_distance(manual_classification, clusterizer.grouping, False)

## k-Means Classic Metrics

In [ ]:
from notebook_widgets import k_means_clustering_experiment_widget

display(k_means_clustering_experiment_widget(classic, spine_metrics, spine_dataset, score_func,
                                             max_num_of_clusters=100, classification=manual_classification,
                                             filename_prefix=classic_prefix))

## k-Means Chord Histograms

In [ ]:
from notebook_widgets import k_means_clustering_experiment_widget

display(k_means_clustering_experiment_widget(chord, spine_metrics, spine_dataset, score_func,
                                             max_num_of_clusters=100, classification=manual_classification,
                                             filename_prefix=chord_prefix))

## DBSCAN Classic Metrics

In [ ]:
from notebook_widgets import dbscan_clustering_experiment_widget

min_eps = 0.2
max_eps = 6
eps_step = 0.1
use_pca = True

display(dbscan_clustering_experiment_widget(classic, spine_metrics, spine_dataset, score_func,
                                            min_eps=min_eps, max_eps=max_eps, eps_step=eps_step, use_pca=use_pca,
                                            classification=manual_classification, filename_prefix=classic_prefix))

## DBSCAN Chord Histograms Euclidean Distance

In [ ]:
from notebook_widgets import dbscan_clustering_experiment_widget

min_eps = 0.1
max_eps = 3
eps_step = 0.1
use_pca = True

display(dbscan_clustering_experiment_widget(chord, spine_metrics, spine_dataset, score_func,
                                            min_eps=min_eps, max_eps=max_eps, eps_step=eps_step, use_pca=use_pca,
                                            classification=manual_classification, filename_prefix=f"{chord_prefix}_euclidean"))

## DBSCAN Chord Histograms Jensen — Shannon Distance

In [ ]:
from notebook_widgets import dbscan_clustering_experiment_widget
from scipy.spatial.distance import jensenshannon
import numpy as np

min_eps = 0.1
max_eps = 1
eps_step = 0.01
use_pca = False

def js_distance(x, y) -> float:
    return np.sqrt(jensenshannon(x, y))

display(dbscan_clustering_experiment_widget(chord, spine_metrics, spine_dataset, score_func, metric=js_distance,
                                            min_eps=min_eps, max_eps=max_eps, eps_step=eps_step, use_pca=use_pca,
                                            classification=manual_classification, filename_prefix=f"{chord_prefix}_jensenshannon"))

## K-Means Spherical Harmonics descriptor

In [7]:
dataset_folder = '0.025-0.025-0.1-dataset'
output_dir = "output/clustering_normalized_shpHarm"
cur_metrics = ["SphericalGarmonics"]
shpHarm_prefix = f"sphHarm"
calculate_metrics = False
save_metrics = True
standardize_metrics = True
scale = (1, 1, 5)

In [8]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import k_means_clustering_experiment_widget, create_dir, SpineMeshDataset

create_dir(output_dir)
spine_dataset = SpineMeshDataset().load(dataset_folder)
spine_dataset.apply_scale(scale)

if calculate_metrics:
    every_spine_metrics = SpineMetricDataset()
    every_spine_metrics.calculate_metrics(spine_dataset.spine_meshes, cur_metrics)

    if save_metrics:
        every_spine_metrics.save(f"{output_dir}/metrics.csv")

every_spine_metrics = SpineMetricDataset.load(f"{output_dir}/metrics.csv")
if standardize_metrics:
    every_spine_metrics.standardize()

sphHarm = every_spine_metrics.get_metrics_subset(cur_metrics)
score_func = lambda clusterizer: 1

In [10]:
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from notebook_widgets import kernel_k_means_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset
from scipy.spatial import distance

# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_folder)
display(k_means_clustering_experiment_widget(sphHarm, every_spine_metrics, spine_dataset, score_func, metric=distance.cityblock,
                                             max_num_of_clusters=5, use_pca=False,
                                             filename_prefix="sphHarm"))

## Kernel k-Means Zernike Moments

In [1]:
dataset_folder = '0.025-0.025-0.1-dataset'
output_dir = "output/clustering_normalized"
cur_metrics = ["LightFieldZernikeMoments"]
zernike_prefix = f"{output_dir}_zernike"
calculate_metrics = False
save_metrics = True
standardize_metrics = True

In [2]:
from spine_analysis.mesh.utils import load_spine_meshes
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from notebook_widgets import k_means_clustering_experiment_widget, create_dir

create_dir(output_dir)
spine_meshes = load_spine_meshes(folder_path=dataset_folder)

if calculate_metrics:
    every_spine_metrics = SpineMetricDataset()
    every_spine_metrics.calculate_metrics(spine_meshes, cur_metrics)

    if save_metrics:
        every_spine_metrics.save(f"{output_dir}/metrics.csv")

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

Could not import submodules (exact error was: numpy.core.multiarray failed to import).

There are many reasons for this error the most common one is that you have
either not built the packages or have built (using `python setup.py build`) or
installed them (using `python setup.py install`) and then proceeded to test
mahotas **without changing the current directory**.

Try installing and then changing to another directory before importing mahotas.
/home/daria/miniconda3/envs/dendritic-spine-shape-analysis/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [3]:
from spine_analysis.shape_metric.io_metric import SpineMetricDataset
from sklearn.metrics import silhouette_score

every_spine_metrics = SpineMetricDataset.load(f"{output_dir}/metrics.csv")
if standardize_metrics:
    every_spine_metrics.standardize()

zernike = every_spine_metrics.get_metrics_subset(['LightFieldZernikeMoments'])
score_func = lambda clusterizer: 1

In [4]:
from spine_analysis.shape_metric import LightFieldZernikeMomentsSpineMetric
from notebook_widgets import kernel_k_means_clustering_experiment_widget
from notebook_widgets import SpineMeshDataset

# load meshes and apply scale
spine_dataset = SpineMeshDataset().load(dataset_folder)
display(kernel_k_means_clustering_experiment_widget(zernike, every_spine_metrics, spine_dataset, score_func, metric=LightFieldZernikeMomentsSpineMetric.repr_distance,
                                             max_num_of_clusters=4, use_pca=False,
                                             filename_prefix="zernike"))

In [19]:
v, f = spine_dataset.get_dendrite_v_f("0.025-0.025-0.1-dataset/1013-2/spine_0.off")

In [23]:
spine_dataset.dendrite_v_f

{'0.025-0.025-0.1-dataset/1009-3\\surface_mesh.off': (array([], shape=(0, 3), dtype=float64),
  array([], shape=(0, 3), dtype=uint64)),
 '0.025-0.025-0.1-dataset/3_full_res (10-2)\\surface_mesh.off': (array([], shape=(0, 3), dtype=float64),
  array([], shape=(0, 3), dtype=uint64)),
 '0.025-0.025-0.1-dataset/1010-1\\surface_mesh.off': (array([], shape=(0, 3), dtype=float64),
  array([], shape=(0, 3), dtype=uint64)),
 '0.025-0.025-0.1-dataset/3_full_res (19)-1\\surface_mesh.off': (array([], shape=(0, 3), dtype=float64),
  array([], shape=(0, 3), dtype=uint64)),
 '0.025-0.025-0.1-dataset/1009-2\\surface_mesh.off': (array([], shape=(0, 3), dtype=float64),
  array([], shape=(0, 3), dtype=uint64)),
 '0.025-0.025-0.1-dataset/3_full_res (1)\\surface_mesh.off': (array([], shape=(0, 3), dtype=float64),
  array([], shape=(0, 3), dtype=uint64)),
 '0.025-0.025-0.1-dataset/3_full_res (4)\\surface_mesh.off': (array([], shape=(0, 3), dtype=float64),
  array([], shape=(0, 3), dtype=uint64)),
 '0.025-0.

## View clusterization

In [ ]:
from ipywidgets import widgets
from notebook_widgets import inspect_grouping_widget


clusterization_path = "output/clusterization/0.025 0.025 0.1 dataset_chord_kmeans_num_of_clusters=6_6_clusters.json"

clusterization = SpineGrouping().load(clusterization_path)

print("Combined Metrics / Classic Metrics / Chord Histograms")
display(widgets.HBox([clusterization.show(spine_metrics), clusterization.show(classic), clusterization.show(chord)]))

display(inspect_grouping_widget(clusterization, spine_dataset, spine_metrics, spine_metrics, manual_classification))